In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'yfinance'

In [7]:
data = yf.download('BTC-USD','2015-01-01','2023-12-17')

NameError: name 'yf' is not defined

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.drop(columns = ['Date','Open','High','Low','Adj Close','Volume'], inplace=True)

In [ ]:
data

In [ ]:
plt.figure(figsize=(8,10))
plt.plot(data)
plt.show()

In [ ]:
train_data = data[:-100]
test_data = data[-100:]

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
train_data_scale = scaler.fit_transform(train_data)

In [ ]:
base_days = 100

In [ ]:
x = []
y = []
for i in range(base_days, train_data_scale.shape[0]):
    x.append(train_data_scale[i-base_days:i])
    y.append(train_data_scale[i,0])

In [ ]:
x, y = np.array(x), np.array(y)

In [ ]:
x = np.reshape(x, (x.shape[0],x.shape[1],1))

In [ ]:
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(50, activation = 'relu', return_sequences=True, input_shape=(x.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(60, activation='relu',return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(80, activation='relu',return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(120, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'mean_squared_error')